In [ ]:
#%pip install yfinance numpy statsmodels pandas matplotlib arch pycop

In [ ]:
%reset -f

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# List of ticker symbols for each index
tickers = ["^GDAXI", "^AEX", "^N225", "^GSPC", "^GSPTSE"]  # DAX, AEX, Nikkei 225, S&P 500, TSX

# Initialize a DataFrame to store merged data
merged_data = pd.DataFrame()

# Loop over each ticker symbol to fetch the data
for ticker in tickers:
    
    
    # Retrieve the historical data starting from November 20, 1992, rescaled by a factor of 0.01
    index = yf.Ticker(ticker)
    index_data = index.history(start="1992-11-20") * 0.01
    
    print(close_data)

    # Extract only the Date and Close columns, reset the index
    close_data = index_data[['Close']].reset_index()
    
    
    # Convert the Date column to Datetime
    close_data['Date'] = pd.to_datetime(close_data['Date'])

    print(close_data)

    #Set Date as index
    close_data.set_index('Date', inplace=True)

    #Sample Data on a Quarterly Basis
    close_data['Close'] = close_data.resample('Q').last()

    #Compute Log Returns
    close_data = np.log(close_data['Close'].pct_change()+1)


    # Rename the columns
    close_data = close_data.rename(columns={'Quarterly Log Return ': ticker})
    
    


    # Merge with the previously merged data
    if merged_data.empty:
        merged_data = close_data
    else:
        merged_data = pd.merge(merged_data, close_data, on='Date', how='outer')



    




Date
1992-11-20 00:00:00+01:00    NaN
1992-11-23 00:00:00+01:00    NaN
1992-11-24 00:00:00+01:00    NaN
1992-11-25 00:00:00+01:00    NaN
1992-11-26 00:00:00+01:00    NaN
                            ... 
2024-11-18 00:00:00+01:00    0.0
2024-11-19 00:00:00+01:00    0.0
2024-11-20 00:00:00+01:00    0.0
2024-11-21 00:00:00+01:00    0.0
2024-11-22 00:00:00+01:00    0.0
Name: Close, Length: 8111, dtype: float64
                          Date       Close
0    1992-11-20 00:00:00+01:00   15.478400
1    1992-11-23 00:00:00+01:00   15.236200
2    1992-11-24 00:00:00+01:00   15.266801
3    1992-11-25 00:00:00+01:00   15.134200
4    1992-11-26 00:00:00+01:00   15.230200
...                        ...         ...
8106 2024-11-18 00:00:00+01:00  191.891895
8107 2024-11-19 00:00:00+01:00  190.603105
8108 2024-11-20 00:00:00+01:00  190.047793
8109 2024-11-21 00:00:00+01:00  191.461699
8110 2024-11-22 00:00:00+01:00  193.225898

[8111 rows x 2 columns]


C:\Users\liamm\AppData\Local\Temp\ipykernel_8404\378253153.py:34: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  close_data['Close'] = close_data.resample('Q').last()
C:\Users\liamm\AppData\Local\Temp\ipykernel_8404\378253153.py:37: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  close_data = np.log(close_data['Close'].pct_change()+1)


TypeError: Series.rename() got an unexpected keyword argument 'columns'

In [ ]:
from arch import arch_model
from scipy.stats import t

#Fit Student's t-GARCH
def fit_garch(ts):
    model = arch_model(ts, vol="Garch", p=1, q=1, dist="t")
    fitted = model.fit(disp="off")
    residuals = ts - fitted.conditional_volatility  # Approximation for simplicity
    nu = fitted.params["nu"]
    return residuals, nu


marginal_results = [fit_garch(merged_data_filled[i]) for i in tickers]
residuals_list, nu_list = zip(*marginal_results)


print(nu_list)





KeyError: '^GDAXI'

In [ ]:
import numpy as np
#import pandas as pd


#Seed
np.random.RandomState = 23456

#Number Monte Carlo Simulations
n = 1000

simulated_data = []

for i in tickers:
    simulated_data.append(multivariate_t.rvs())

In [ ]:







# Step 1: Transform the data to uniform margins using ECDF (Empirical CDF)
uniform_data = pd.DataFrame()


# Use empirical CDF for each column (ticker) in the merged data
for ticker in tickers:
    uniform_data[ticker] = merged_data_filled[ticker].rank() / len(merged_data_filled[ticker])

# Step 2: Fit a copula (using Gaussian copula as an example)
copula = StudentTCopula()

# Step 3: Fit the copula to the uniform-transformed data
copula.fit(uniform_data)

# Step 4: Simulate synthetic data from the fitted copula (optional)
simulated_data = copula.sample(len(merged_data_filled))

# Convert the uniform data back to the original scale (inverse of the transformation)
simulated_data_original_scale = pd.DataFrame()

for idx, ticker in enumerate(tickers):
    # Inverse transform (since we used ECDF, we can approximate the inverse by using the quantiles)
    simulated_data_original_scale[ticker] = np.percentile(merged_data_filled[ticker], simulated_data[:, idx] * 100)

# View the simulated data
print(simulated_data_original_scale.head())


In [ ]:
%pip install pycop

Note: you may need to restart the kernel to use updated packages.
